In [3]:
from unittest.mock import DEFAULT
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import sys
import configparser
import pandas as pd

#? ---configparserの宣言 iniファイルの読み込み---
#* configのディレクトリを記入する .read('●●●●●●●●' , encording='utf-8')
config_ini = configparser.ConfigParser()
config_ini.read('baba_condition_config.ini', encoding='utf-8')

#? -----定数定義-----
#* config.ini内で指定している
# webdriverのパス
driver = webdriver.Chrome(service=Service(config_ini.get('DEFAULT', 'DRIVER')))
# 馬場状態を保存するDBcsvのパス/ファイル名
baba_condition_DB_filepath = config_ini.get('DEFAULT', 'DB_FILE_PATH')

#? -----関数定義-----
#* 対象のxpathの要素を探してテキストにする関数を定義する
# nameは変数名、xpathは取得したいパス
def to_text(name,xpath):
    name = driver.find_element(By.XPATH, xpath)
    name = name.text
    return name

# URLを読み込んで馬場情報を取得する関数
# return list
def get_baba_condition(url):
    driver.get(url)
    print(url , "読み込み")

    # 明示的な待機
    driver.implicitly_wait(20)

    # 空のリストを作成しておく
    list = []

    # 競馬場名
    # 第1回中山競馬第2日前日
    # 中山だけを抜き出す
    name = driver.find_element(By.XPATH, '//*[@id="condition"]/div[1]/div/div/h2')
    name = name.text
    index1 = name.find('回')
    index2 = name.find('競')
    name = name[index1+1:index2]
    list.append(name)

    # 開催日 E.g)（2022年1月7日（金曜））
    name = driver.find_element(By.XPATH, '//*[@id="condition"]/div[1]/div/div/h2/span')
    name = name.text
    index1 = name.find('（')
    index2 = name.find('日')
    name = name[index1+1:index2+1]
    list.append(name)

    # クッション値
    list.append(to_text('cushion_value', '//*[@id="cushion_num"]/p/strong'))
    # 含水率/芝-ゴール前
    list.append(to_text('wc_grass_goal', '//*[@id="turf_line"]/td[1]'))
    # 含水率/芝-4コーナー
    list.append(to_text('wc_grass_4corner', '//*[@id="turf_line"]/td[2]'))
    # 含水率/ダート-ゴール前
    list.append(to_text('wc_dirt_goal', '//*[@id="dirt_line"]/td[1]'))
    # 含水率/ダート-4コーナー
    list.append(to_text('wc_dirt_4corner', '//*[@id="dirt_line"]/td[2]'))

    print(list)

    return list

#* 取得したlistをDBにいれる
def input_to_DB(list):
    # DBをデータフレーム型に格納
    df1 = pd.DataFrame(pd.read_csv(baba_condition_DB_filepath, encoding='s'))

    # 取得したリストをdf形式にする
    #! 一旦2次元配列にしないと1行のdfとして変換されない
    lst2D = []
    lst2D.append(list)
    input_df = pd.DataFrame(lst2D, columns=['keibajo', 'date', 'cushion_value', 'mc_dirt_4corner', 'mc_dirt_goal', 'mc_turf_4corner', 'mc_turf_goal'])

    # DBと取得したcsvを結合する
    df = df1.append(input_df,ignore_index=True)

    #重複を削除
    sindf = df.drop_duplicates(subset=['keibajo', 'date'])
    sindf

    # CSVで保存
    sindf.to_csv(baba_condition_DB_filepath, index=False, encoding='utf-8')

input_to_DB(get_baba_condition(config_ini.get('DEFAULT', 'URL1')))
input_to_DB(get_baba_condition(config_ini.get('DEFAULT', 'URL2')))
input_to_DB(get_baba_condition(config_ini.get('DEFAULT', 'URL3')))

driver.quit()

#* プログラムの停止
sys.exit()



https://www.jra.go.jp/keiba/baba/index.html 読み込み
['東京', '2022年5月21日', '9.4', '14.7%', '13.3%', '4.0%', '3.9%']
https://www.jra.go.jp/keiba/baba/index2.html 読み込み
['中京', '2022年5月21日', '9.5', '12.3%', '11.6%', '5.5%', '4.8%']
https://www.jra.go.jp/keiba/baba/index3.html 読み込み
['新潟', '2022年5月21日', '9.7', '10.8%', '11.0%', '4.6%', '3.8%']


SystemExit: 